In [39]:
import numpy as np
import pandas as pd

import sklearn.datasets as sd
import sklearn.feature_extraction.text as ft
import sklearn.model_selection as ms
import sklearn.linear_model as lm
import sklearn.tree as st
import sklearn.ensemble as se
import sklearn.naive_bayes as nb
import sklearn.svm as svm
import sklearn.metrics as sm

import nltk.tokenize as tk
import jieba
jieba.load_userdict("mydict.txt")

In [40]:
data = pd.read_csv("htl_all.csv")
data["label"].value_counts()
data.columns

Index(['label', 'review'], dtype='object')

In [41]:
def func(item):
    # 分词
    seg_list = jieba.cut(item)
    return " ".join(seg_list)



data["new_reviews"] = data["review"].apply(func)
data = data.tail((data["label"]==0).sum()*2)

data["label"].value_counts()

1    2443
0    2443
Name: label, dtype: int64

In [42]:
# 整理输入集与输出集
cv = ft.CountVectorizer()
bow = cv.fit_transform(data["new_reviews"]) #按空格间隔词的形式作为输入，可转换为词袋矩阵bow

tt = ft.TfidfTransformer()
tfidf = tt.fit_transform(bow)



# 拆分测试集与训练集
train_x,test_x,train_y,test_y=\
    ms.train_test_split(tfidf,data["label"],test_size=0.1,stratify=data["label"],random_state=7)





# 交叉验证
# model_NB = lm.LogisticRegression()
# model_NB = svm.SVC(kernel="linear",C=1,probability=True) #再SVM模型中，设置probability用于之后可以输出置信概率矩阵，其他模型不需要设置该项
# model_NB = st.DecisionTreeClassifier(max_depth=10,min_samples_split=3,class_weight="balanced",random_state=7) 
#不少模型通过设置class_weighted参数来弥补不同类别样本数量不平衡的问题，朴素贝叶斯模型无此参数！
# model_NB = se.RandomForestClassifier(max_depth=10,min_samples_split=3,n_estimators=200,random_state=7)

# score_f1 = ms.cross_val_score(model_NB,train_x,train_y,cv=5,scoring="f1_weighted")
# print(score_f1.mean())


#使用朴素贝叶斯模型
model_NB = nb.MultinomialNB()
score_f1 = ms.cross_val_score(model_NB,train_x,train_y,cv=5,scoring="f1_weighted")
print(score_f1.mean())



#训练模型
model_NB.fit(train_x,train_y)


#测试模型，评估模型
pred_test_y = model_NB.predict(test_x)
print(sm.classification_report(test_y,pred_test_y))



0.862797500948426
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       244
           1       0.89      0.87      0.88       245

    accuracy                           0.88       489
   macro avg       0.88      0.88      0.88       489
weighted avg       0.88      0.88      0.88       489



In [51]:
new_comment=["房间很宽敞，价格不贵",
             "睡得不舒服，早餐太贵且不好吃,差评",
             "总体还行，考虑到携程上有红包优惠",
             "设施不齐全，下次再也来了。",
             "地板脏，空调漏水，下次不会再来了！",
             "一般，凑合，马马虎虎"]

new_comment = pd.DataFrame(new_comment,columns=["reviews"])
new_comment["seg"] = new_comment["reviews"].apply(func)
new_comment

,reviews,seg
0,房间很宽敞，价格不贵,房间 很 宽敞 ， 价格 不 贵
1,"睡得不舒服，早餐太贵且不好吃,差评","睡得 不舒服 ， 早餐 太贵 且 不好吃 , 差评"
2,总体还行，考虑到携程上有红包优惠,总体 还 行 ， 考虑 到 携程 上 有 红包 优惠
3,设施不齐全，下次再也来了。,设施 不齐全 ， 下次 再也 来 了 。
4,地板脏，空调漏水，下次不会再来了！,地板 脏 ， 空调 漏水 ， 下次 不会 再来 了 ！
5,一般，凑合，马马虎虎,一般 ， 凑合 ， 马马虎虎


In [54]:
# 把样本按照训练时 的方式转换为tfidf矩阵，才可以交给模型训练
bow = cv.transform(new_comment["seg"])
test_data = tt.transform(bow)

pred_test_data = model_NB.predict(test_data)

#输出置信概率
pred_prob_y = model_NB.predict_proba(test_data)

for b,c in zip(pred_test_y,pred_prob_y):
    print(b,":",c)

1 : [0.1479043 0.8520957]
1 : [0.57944012 0.42055988]
0 : [0.39309168 0.60690832]
1 : [0.70030885 0.29969115]
0 : [0.79498323 0.20501677]
1 : [0.60479671 0.39520329]
